In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import keras
import scipy as sp
import scipy.signal
import tensorflow as tf
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers
import scipy.io as sio
import numpy as np
import sklearn.metrics as metrics 
import matplotlib.pyplot as plt
import statistics

from tensorflow_addons.metrics import RSquare
from tensorflow.keras.models import Sequential
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from wandb.keras import WandbCallback
import wandb
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
F1_test=8
D_test=2
F2_test=16
Drop_test=0 #originally 0.5
KernLength_test=64 #Originally 64 
batch_test=16

N_chan=31
N_samples_long=250
# N_samples_short=251
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# PER SUB 
# ########SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
SUBJECTS=[9]
for temp_sub in SUBJECTS:
    CLASSES= [50]
    #questa va applicata a for loop di subject che deve essere il più esterno 
    sub="{:02d}".format(temp_sub)

    dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_sub/output", "sub_"+str(sub))
#     os.mkdir(dir00)
    for n_classes in CLASSES:
        #Define loss function
        loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        LABELS= list(range(0,n_classes))
        
        if n_classes==50:
            numero_classi=5
        else:
            numero_classi=n_classes
        
        evaluation=[]
        iteration=[]
        confusion_matrix_x_test=[]
        confusion_matrix_y_test= []
        validation_acc=[]
        PERFORMANCE=[]

        print("SUBJECT: "+ str(sub))
        print("N_CLASSES: "+ str(n_classes))
        
        X=np.load("input/sub_"+str(sub)+ "_compiled_RAW_downsampled_float32.npy")
        y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_sub/input/sub-"+str(sub)+"_compiled_rightSMA_scaled_per_sub_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels"]

        X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
        y=y.flatten()
        print("New shape for X: " + str(X.shape))
        print("New shape for y: "+str(y.shape))

        dir0= os.path.join(dir00, str(n_classes)+"_class")
#         os.mkdir(dir0)
        dir1=os.path.join(dir0, "comparison")
        dir2=os.path.join(dir0, "temporal_convolution")
        dir3=os.path.join(dir0, "spatial_convolution")
#         os.mkdir(dir1)
#         os.mkdir(dir2)
#         os.mkdir(dir3)

        ################################################################################################################################################################################################################################################################################

        n_folds = 5
        seed = 21
        shuffle_test = True
        EPOCHS=250

        kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
        count=0
        sommatoria=0

        for train_index, test_index in kfold.split(X):
            count=count+1

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            print("Train index for this split: "+ str(train_index)) 
            print("Number of samples for train set: "+str(train_index.shape[0]))
            print("Test index for this split: "+ str(test_index))
            print("Number of samples for test set: "+str(test_index.shape[0]))

            # Define the model architecture - 

            model=Sequential()

            ##################################################################

            model.add(Conv2D(8, (1, 64), padding = 'same',
                                           input_shape = (N_chan, N_samples_long, 1),
                                           use_bias = False, name="temporal_conv"))
            model.add(BatchNormalization(name="batchnorm_1"))
            model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                           depth_multiplier = 2,
                                           depthwise_constraint = max_norm(1.), name="spatial_conv"))
            model.add(BatchNormalization(name="batchnorm_2"))
            model.add(Activation('elu', name="activation_1"))
            model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
            model.add(Dropout(0.5, name="dropout_1"))

            model.add(SeparableConv2D(16, (1, 16),
                                           use_bias = False, padding = 'same', name="separable_conv"))
            model.add(BatchNormalization(name="batchnorm_3"))
            model.add(Activation('elu', name="activation_2"))
            model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
            model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

            model.add(Flatten(name = 'flatten'))

            model.add(Dense(numero_classi, name = 'dense', 
                                     kernel_constraint = max_norm(0.25)))
            model.add(Activation('softmax', name = 'softmax'))


            # Define the optimizer
            optimizer= optimizers.Adam(
            learning_rate= 1e-4,
            weight_decay= 0
            )
            model.compile(optimizer=optimizer,
                           loss=loss_fn,
                           metrics=['accuracy'])

            evaluation.append(model.fit(X_train, y_train, batch_size=16,
                      epochs=EPOCHS, 
                      validation_data=(X_test, y_test), 
                      verbose=0, workers=1)
                         )

            # Iteration = fold, i am just saving the model for that fold
            iteration.append(model)

            confusion_matrix_x_test.append(X_test)
            confusion_matrix_y_test.append(y_test)

            #Plotting confusion matrix
            pred=model.predict(X_test)
            y_test_pred= np.argmax(pred, axis=1)

            confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
            plt.figure()
            metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
            plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
            plt.close()

            validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

            PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

            #Salvo risultati di singolo fold
            sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
            sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


            # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
            var= (model.get_layer("temporal_conv").weights)
            for lallo in range(8):
                plt.figure()
                plt.title("temp_conv_"+str(lallo))
                plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
                plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
                temp= (var[0][0,:,0][:,lallo]).numpy()
                sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
                plt.close()


            var_2= (model.get_layer("spatial_conv").weights)
            reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
            for lallo in range(16):
                nump= reshaped_var_2[:,lallo].numpy()
                sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

        ###################################################################################################################

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
        min_temp_loss=10
        min_temp_acc=10
        max_temp_loss=0
        max_temp_acc=0

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['loss'])
            if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['val_loss'])
            if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['loss'])
            if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['val_loss'])

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['accuracy'])
            if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
            if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['accuracy'])
            if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['loss']
            loss_vec_test= evaluation[idx].history['val_loss']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Loss across epochs for fold: '+str(idx))
            plt.ylim([min_temp_loss, max_temp_loss])
            plt.legend()

            plt.savefig(dir1+"/loss_kfold_"+str(idx))
            plt.close()

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['accuracy']
            loss_vec_test= evaluation[idx].history['val_accuracy']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title('Accuracy across epochs for fold: '+str(idx))
            plt.ylim([min_temp_acc, max_temp_acc])
            plt.legend()

            plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
            plt.close()

        #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
        accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
        accuratezza[0]=(np.mean(validation_acc))
        accuratezza[1]=(statistics.pstdev(validation_acc))


        precisione=[]
        recall=[]
        f1_score=[]
        support=[]
        for classe in range(n_classes):
            precision_temp=[]
            recall_temp=[]
            f1_score_temp=[]
            support_temp=[]
            for idx in range(n_folds):
                precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
                recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
                f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
                support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

            precisione.append(np.mean(precision_temp))
            precisione.append(statistics.pstdev(precision_temp))
            recall.append(np.mean(recall_temp))
            recall.append(statistics.pstdev(recall_temp))
            f1_score.append(np.mean(f1_score_temp))
            f1_score.append(statistics.pstdev(f1_score_temp))    
            support.append(np.mean(support_temp))
            support.append(statistics.pstdev(support_temp)) 

        sommario=[]
        sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
        sio.savemat(dir0+"/sommario.mat", {"array": sommario})

        gianfranco=model.predict(X)
        gianfranco2=np.argmax(gianfranco, axis=1)
        sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



In [ ]:
# PER SUB & PROTOCOL
SUBJECTS=[6,7,8,9,11,12,13,14,16,17]
for temp_sub in SUBJECTS:
    CLASSES= [2,3,5,10,50]
    #questa va applicata a for loop di subject che deve essere il più esterno 
    sub="{:02d}".format(temp_sub)

    dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_sub_protocol/output", "sub_"+str(sub))
    os.mkdir(dir00)
    for n_classes in CLASSES:
        #Define loss function
        loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        LABELS= list(range(0,n_classes))
        
        if n_classes==50:
            numero_classi=5
        else:
            numero_classi=n_classes

        evaluation=[]
        iteration=[]
        confusion_matrix_x_test=[]
        confusion_matrix_y_test= []
        validation_acc=[]
        PERFORMANCE=[]

        print("SUBJECT: "+ str(sub))
        print("N_CLASSES: "+ str(n_classes))
        
        X=np.load("input/sub_"+str(sub)+ "_compiled_RAW_downsampled_float32.npy")
        y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_sub_protocol/input/sub-"+str(sub)+"_compiled_rightSMA_scaled_per_sub_protocol_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels"]

        X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
        y=y.flatten()
        print("New shape for X: " + str(X.shape))
        print("New shape for y: "+str(y.shape))

        dir0= os.path.join(dir00, str(n_classes)+"_class")
        os.mkdir(dir0)
        dir1=os.path.join(dir0, "comparison")
        dir2=os.path.join(dir0, "temporal_convolution")
        dir3=os.path.join(dir0, "spatial_convolution")
        os.mkdir(dir1)
        os.mkdir(dir2)
        os.mkdir(dir3)

        ################################################################################################################################################################################################################################################################################

        n_folds = 5
        seed = 24
        shuffle_test = True
        EPOCHS=250

        kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
        count=0
        sommatoria=0

        for train_index, test_index in kfold.split(X):
            count=count+1

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            print("Train index for this split: "+ str(train_index)) 
            print("Number of samples for train set: "+str(train_index.shape[0]))
            print("Test index for this split: "+ str(test_index))
            print("Number of samples for test set: "+str(test_index.shape[0]))

            # Define the model architecture - 

            model=Sequential()

            ##################################################################

            model.add(Conv2D(8, (1, 64), padding = 'same',
                                           input_shape = (N_chan, N_samples_long, 1),
                                           use_bias = False, name="temporal_conv"))
            model.add(BatchNormalization(name="batchnorm_1"))
            model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                           depth_multiplier = 2,
                                           depthwise_constraint = max_norm(1.), name="spatial_conv"))
            model.add(BatchNormalization(name="batchnorm_2"))
            model.add(Activation('elu', name="activation_1"))
            model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
            model.add(Dropout(0.5, name="dropout_1"))

            model.add(SeparableConv2D(16, (1, 16),
                                           use_bias = False, padding = 'same', name="separable_conv"))
            model.add(BatchNormalization(name="batchnorm_3"))
            model.add(Activation('elu', name="activation_2"))
            model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
            model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

            model.add(Flatten(name = 'flatten'))

            model.add(Dense(numero_classi, name = 'dense', 
                                     kernel_constraint = max_norm(0.25)))
            model.add(Activation('softmax', name = 'softmax'))


            # Define the optimizer
            optimizer= optimizers.Adam(
            learning_rate= 1e-4,
            weight_decay= 0
            )
            model.compile(optimizer=optimizer,
                           loss=loss_fn,
                           metrics=['accuracy'])

            evaluation.append(model.fit(X_train, y_train, batch_size=16,
                      epochs=EPOCHS, 
                      validation_data=(X_test, y_test), 
                      verbose=0, workers=1)
                         )

            # Iteration = fold, i am just saving the model for that fold
            iteration.append(model)

            confusion_matrix_x_test.append(X_test)
            confusion_matrix_y_test.append(y_test)

            #Plotting confusion matrix
            pred=model.predict(X_test)
            y_test_pred= np.argmax(pred, axis=1)

            confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
            plt.figure()
            metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
            plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
            plt.close()

            validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

            PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

            #Salvo risultati di singolo fold
            sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
            sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


            # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
            var= (model.get_layer("temporal_conv").weights)
            for lallo in range(8):
                plt.figure()
                plt.title("temp_conv_"+str(lallo))
                plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
                plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
                temp= (var[0][0,:,0][:,lallo]).numpy()
                sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
                plt.close()


            var_2= (model.get_layer("spatial_conv").weights)
            reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
            for lallo in range(16):
                nump= reshaped_var_2[:,lallo].numpy()
                sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

        ###################################################################################################################

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
        min_temp_loss=10
        min_temp_acc=10
        max_temp_loss=0
        max_temp_acc=0

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['loss'])
            if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['val_loss'])
            if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['loss'])
            if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['val_loss'])

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['accuracy'])
            if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
            if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['accuracy'])
            if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['loss']
            loss_vec_test= evaluation[idx].history['val_loss']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Loss across epochs for fold: '+str(idx))
            plt.ylim([min_temp_loss, max_temp_loss])
            plt.legend()

            plt.savefig(dir1+"/loss_kfold_"+str(idx))
            plt.close()

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['accuracy']
            loss_vec_test= evaluation[idx].history['val_accuracy']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title('Accuracy across epochs for fold: '+str(idx))
            plt.ylim([min_temp_acc, max_temp_acc])
            plt.legend()

            plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
            plt.close()

        #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
        accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
        accuratezza[0]=(np.mean(validation_acc))
        accuratezza[1]=(statistics.pstdev(validation_acc))


        precisione=[]
        recall=[]
        f1_score=[]
        support=[]
        for classe in range(n_classes):
            precision_temp=[]
            recall_temp=[]
            f1_score_temp=[]
            support_temp=[]
            for idx in range(n_folds):
                precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
                recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
                f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
                support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

            precisione.append(np.mean(precision_temp))
            precisione.append(statistics.pstdev(precision_temp))
            recall.append(np.mean(recall_temp))
            recall.append(statistics.pstdev(recall_temp))
            f1_score.append(np.mean(f1_score_temp))
            f1_score.append(statistics.pstdev(f1_score_temp))    
            support.append(np.mean(support_temp))
            support.append(statistics.pstdev(support_temp)) 

        sommario=[]
        sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
        sio.savemat(dir0+"/sommario.mat", {"array": sommario})

        gianfranco=model.predict(X)
        gianfranco2=np.argmax(gianfranco, axis=1)
        sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



In [ ]:
# PER RUN 
# SUBJECTS=[3,4,5,6,7,8,9,11,12,13,14,16,17]
SUBJECTS=[17]
for temp_sub in SUBJECTS:
#     CLASSES= [2,3,5,10,50]
    CLASSES= [3,5,10,50]
    #questa va applicata a for loop di subject che deve essere il più esterno 
    sub="{:02d}".format(temp_sub)

    dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_run/output", "sub_"+str(sub))
#     os.mkdir(dir00)
    for n_classes in CLASSES:
        #Define loss function
        loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        LABELS= list(range(0,n_classes))
        
        if n_classes==50:
            numero_classi=5
        else:
            numero_classi=n_classes

        evaluation=[]
        iteration=[]
        confusion_matrix_x_test=[]
        confusion_matrix_y_test= []
        validation_acc=[]
        PERFORMANCE=[]

        print("SUBJECT: "+ str(sub))
        print("N_CLASSES: "+ str(n_classes))
        
        X=np.load("input/sub_"+str(sub)+ "_compiled_RAW_downsampled_float32.npy")
        y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_run/input/sub-"+str(sub)+"_compiled_rightSMA_scaled_per_run_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels"]

        X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
        y=y.flatten()
        print("New shape for X: " + str(X.shape))
        print("New shape for y: "+str(y.shape))

        dir0= os.path.join(dir00, str(n_classes)+"_class")
        os.mkdir(dir0)
        dir1=os.path.join(dir0, "comparison")
        dir2=os.path.join(dir0, "temporal_convolution")
        dir3=os.path.join(dir0, "spatial_convolution")
        os.mkdir(dir1)
        os.mkdir(dir2)
        os.mkdir(dir3)

        ################################################################################################################################################################################################################################################################################

        n_folds = 5
#         seed = 21
        shuffle_test = True
        EPOCHS=250

        kfold = KFold(n_splits = n_folds, shuffle = shuffle_test)
        count=0
        sommatoria=0

        for train_index, test_index in kfold.split(X):
            count=count+1

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            print("Train index for this split: "+ str(train_index)) 
            print("Number of samples for train set: "+str(train_index.shape[0]))
            print("Test index for this split: "+ str(test_index))
            print("Number of samples for test set: "+str(test_index.shape[0]))

            # Define the model architecture - 

            model=Sequential()

            ##################################################################

            model.add(Conv2D(8, (1, 64), padding = 'same',
                                           input_shape = (N_chan, N_samples_long, 1),
                                           use_bias = False, name="temporal_conv"))
            model.add(BatchNormalization(name="batchnorm_1"))
            model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                           depth_multiplier = 2,
                                           depthwise_constraint = max_norm(1.), name="spatial_conv"))
            model.add(BatchNormalization(name="batchnorm_2"))
            model.add(Activation('elu', name="activation_1"))
            model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
            model.add(Dropout(0.5, name="dropout_1"))

            model.add(SeparableConv2D(16, (1, 16),
                                           use_bias = False, padding = 'same', name="separable_conv"))
            model.add(BatchNormalization(name="batchnorm_3"))
            model.add(Activation('elu', name="activation_2"))
            model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
            model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

            model.add(Flatten(name = 'flatten'))

            model.add(Dense(numero_classi, name = 'dense', 
                                     kernel_constraint = max_norm(0.25)))
            model.add(Activation('softmax', name = 'softmax'))


            # Define the optimizer
            optimizer= optimizers.Adam(
            learning_rate= 1e-4,
            weight_decay= 0
            )
            model.compile(optimizer=optimizer,
                           loss=loss_fn,
                           metrics=['accuracy'])

            evaluation.append(model.fit(X_train, y_train, batch_size=16,
                      epochs=EPOCHS, 
                      validation_data=(X_test, y_test), 
                      verbose=0, workers=1)
                         )

            # Iteration = fold, i am just saving the model for that fold
            iteration.append(model)

            confusion_matrix_x_test.append(X_test)
            confusion_matrix_y_test.append(y_test)

            #Plotting confusion matrix
            pred=model.predict(X_test)
            y_test_pred= np.argmax(pred, axis=1)

            confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
            plt.figure()
            metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
            plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
            plt.close()

            validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

            PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

            #Salvo risultati di singolo fold
            sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
            sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


            # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
            var= (model.get_layer("temporal_conv").weights)
            for lallo in range(8):
                plt.figure()
                plt.title("temp_conv_"+str(lallo))
                plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
                plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
                temp= (var[0][0,:,0][:,lallo]).numpy()
                sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
                plt.close()


            var_2= (model.get_layer("spatial_conv").weights)
            reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
            for lallo in range(16):
                nump= reshaped_var_2[:,lallo].numpy()
                sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

        ###################################################################################################################

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
        min_temp_loss=10
        min_temp_acc=10
        max_temp_loss=0
        max_temp_acc=0

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['loss'])
            if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['val_loss'])
            if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['loss'])
            if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['val_loss'])

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['accuracy'])
            if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
            if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['accuracy'])
            if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['loss']
            loss_vec_test= evaluation[idx].history['val_loss']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Loss across epochs for fold: '+str(idx))
            plt.ylim([min_temp_loss, max_temp_loss])
            plt.legend()

            plt.savefig(dir1+"/loss_kfold_"+str(idx))
            plt.close()

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['accuracy']
            loss_vec_test= evaluation[idx].history['val_accuracy']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title('Accuracy across epochs for fold: '+str(idx))
            plt.ylim([min_temp_acc, max_temp_acc])
            plt.legend()

            plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
            plt.close()

        #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
        accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
        accuratezza[0]=(np.mean(validation_acc))
        accuratezza[1]=(statistics.pstdev(validation_acc))


        precisione=[]
        recall=[]
        f1_score=[]
        support=[]
        for classe in range(numero_classi):
            precision_temp=[]
            recall_temp=[]
            f1_score_temp=[]
            support_temp=[]
            for idx in range(n_folds):
                precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
                recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
                f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
                support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

            precisione.append(np.mean(precision_temp))
            precisione.append(statistics.pstdev(precision_temp))
            recall.append(np.mean(recall_temp))
            recall.append(statistics.pstdev(recall_temp))
            f1_score.append(np.mean(f1_score_temp))
            f1_score.append(statistics.pstdev(f1_score_temp))    
            support.append(np.mean(support_temp))
            support.append(statistics.pstdev(support_temp)) 

        sommario=[]
        sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
        sio.savemat(dir0+"/sommario.mat", {"array": sommario})

        gianfranco=model.predict(X)
        gianfranco2=np.argmax(gianfranco, axis=1)
        sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



# ALL-per SUB

In [ ]:
# PER SUB 
# ########SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
# SUBJECTS=[11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
CLASSES= [2,3,5,10,50]
#questa va applicata a for loop di subject che deve essere il più esterno 
#     sub="{:02d}".format(temp_sub)

dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_sub/output", "all")
os.mkdir(dir00)
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))

    if n_classes==50:
        numero_classi=5
    else:
        numero_classi=n_classes

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

    X=np.load("input/RAW_regression_all_subs_float32.npy")
    y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_sub/input/Compiled_rightSMA_all_scaled_per_sub_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels_all"]

    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("Shape for X: " + str(X.shape))
    print("Shape for y: "+str(y.shape))

    X= X[762:,:,:,:]
    y= y[762:]
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))
    
    dir0= os.path.join(dir00, str(n_classes)+"_class")
    os.mkdir(dir0)
    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
    seed = 21
    shuffle_test = True
    EPOCHS=250
    DROPOUT_TEST=0.25

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(DROPOUT_TEST, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-4,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=2, workers=1)
                     )

        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 

        sio.savemat(dir0+"/precisione.mat", {"array":precisione})
        sio.savemat(dir0+"/recall.mat", {"array":recall})
        sio.savemat(dir0+"/f1_score.mat", {"array":f1_score})
        sio.savemat(dir0+"/support.mat", {"array":support})

#     sommario=[]
#     sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
#     sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



In [ ]:
# PER SUB & PROTOCOL
# SUBJECTS=[6,7,8,9,11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
CLASSES= [2,3,5,10,50]
#questa va applicata a for loop di subject che deve essere il più esterno 
#     sub="{:02d}".format(temp_sub)

dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_sub_protocol/output", "all")
os.mkdir(dir00)
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))

    if n_classes==50:
        numero_classi=5
    else:
        numero_classi=n_classes

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

#     print("SUBJECT: "+ str(sub))
    print("N_CLASSES: "+ str(n_classes))

    X=np.load("input/RAW_regression_all_subs_float32.npy")
    y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_sub_protocol/input/Compiled_rightSMA_all_scaled_per_sub_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels_all"]

    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("Shape for X: " + str(X.shape))
    print("Shape for y: "+str(y.shape))

    X= X[762:,:,:,:]
    y= y[762:]
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir0= os.path.join(dir00, str(n_classes)+"_class")
    os.mkdir(dir0)
    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
    seed = 24
    shuffle_test = True
    EPOCHS=250
    DROPOUT_TEST=0.25
    
    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(DROPOUT_TEST, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-4,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=0, workers=1)
                     )

        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 

        sio.savemat(dir0+"/precisione.mat", {"array":precisione})
        sio.savemat(dir0+"/recall.mat", {"array":recall})
        sio.savemat(dir0+"/f1_score.mat", {"array":f1_score})
        sio.savemat(dir0+"/support.mat", {"array":support})

#     sommario=[]
#     sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
#     sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



In [ ]:
# PER RUN 
# SUBJECTS=[3,4,5,6,7,8,9,11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
CLASSES= [2,3,5,10,50]
#questa va applicata a for loop di subject che deve essere il più esterno 
# sub="{:02d}".format(temp_sub)

dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_run/output", "all")
os.mkdir(dir00)
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))

    if n_classes==50:
        numero_classi=5
    else:
        numero_classi=n_classes

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

    X=np.load("input/RAW_regression_all_subs_float32.npy")
    y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_run/input/Compiled_rightSMA_all_scaled_per_sub_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels_all"]

    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("Shape for X: " + str(X.shape))
    print("Shape for y: "+str(y.shape))

    X= X[762:,:,:,:]
    y= y[762:]
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir0= os.path.join(dir00, str(n_classes)+"_class")
    os.mkdir(dir0)
    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
#         seed = 21
    shuffle_test = True
    EPOCHS=250
    DROPOUT_TEST=0.25

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(DROPOUT_TEST, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-4,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=0, workers=1)
                     )

        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 
        
        sio.savemat(dir0+"/precisione.mat", {"array":precisione})
        sio.savemat(dir0+"/recall.mat", {"array":recall})
        sio.savemat(dir0+"/f1_score.mat", {"array":f1_score})
        sio.savemat(dir0+"/support.mat", {"array":support})

#     sommario=[]
#     sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
#     sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})

